In [1]:
!./install.sh

   Compiling cleon v0.1.0 (/Users/madhavajay/dev/ladon)
    Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 10.58s
🍹 Building a mixed python/rust project
🔗 Found pyo3 bindings with abi3 support
🐍 Found CPython 3.13 at /Users/madhavajay/dev/ladon/.venv/bin/python
📡 Using build options features from pyproject.toml
💻 Using `MACOSX_DEPLOYMENT_TARGET=11.0` for aarch64-apple-darwin by default
    Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.02s
📦 Including files matching "src/cleon/py.typed"
📦 Including files matching "src/cleon/_cleon.pyi"
📦 Built wheel for abi3 Python ≥ 3.9 to /Users/madhavajay/dev/ladon/dist/cleon-0.1.0-cp39-abi3-macosx_11_0_arm64.whl
Resolved 17 packages in 79ms                                         
Prepared 17 packages in 4ms                                              
Uninstalled 17

In [2]:
import cleon

In [3]:
cleon.use(
    "codex",
    debug=False,
    context_changes=True,
    log_path="./cleon.log",
    context_cells=None
)

Registered %codex cell magic.


<function cleon.magic.register_magic.<locals>._codex_magic(line: 'str', cell: 'str | None' = None) -> 'Any'>

In [4]:
print("hello banana")

hello banana


In [5]:
%history

!./install.sh
import cleon
cleon.use(
    "codex",
    debug=False,
    context_changes=True,
    log_path="./cleon.log",
    context_cells=None
)
print("hello banana")
%history


In [6]:
%%codex
who did i just say hello to?

You printed a greeting to “banana.”

In [ ]:
%%codex
can you clone this repo into /tmp https://github.com/karpathy/micrograd and then read the contents

In [ ]:
%%codex
okay can you read all the code at /tmp/micrograd

In [ ]:
%%codex
give me the first step in a tutorial explaining me how to write my own micrograd from scratch

In [ ]:
from dataclasses import dataclass

In [ ]:
from typing import Tuple

In [ ]:
from dataclasses import field

In [ ]:
from collections.abc import Callable

In [ ]:
@dataclass
class Value:
    data: float = None
    parent: Tuple = field(default_factory=tuple)
    grad: float = 0.0
    _op: str = ""
    _backward: Callable = field(default_factory=lambda: lambda: None)

    def __repr__(self):
        return f"<{self._op}: {self.data}>"

In [ ]:
%%codex
what do you think of my first code?

In [ ]:
%%codex
maybe start with some more building of connected examples in a mini tutorial before explaining why i need to make parent a tuple etc

In [ ]:
a = Value(2.0, parent=(), _op='leaf')
b = Value(3.0, parent=(), _op='leaf')
c = Value(a.data + b.data, parent=(a, b), _op='+')
print(a, b, c)  # <leaf: 2.0> <leaf: 3.0> <+: 5.0>

In [ ]:
%%codex
okay how does that look?